# Finetune: figuring out why original code not recording f1 in history.csv

Copy of `jupyter_notebooks/finetune_train_random_original.ipynb`,

but now we try adding macro f1 metric to the model metrics to be evaluated on the validation set at epoch end. The goal is to get macro f1 scores into the output history.csv so that we can plot the validation score by epoch curve. This would be our attempt to reproduce Figure 3 in the paper.

---

Original prepared dataset is the code in `finetuning/readme.md` which samples at 250 hz, 65 seconds.

Now we try training on an uninitialized CNN.

We do this by *not* passing in the `--weights-file` parameter to `finetuning.trainer`.

In [1]:
# You may also manually mount drive by clicking on folder icon in left sidebar
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PROJECT_ROOT = '/content/drive/MyDrive/DLHProject'

If you have colab pro, clone our repo to /root directory.

Sample commands to run in Colab Pro Terminal:

```bash
$ cd /root
# enter your username and github PAT
$ git clone https://github.com/myles-i/DLH_TransferLearning.git
$ cd DLH_TransferLearning
```

In [3]:
# REPO = PROJECT_ROOT + '/Danielgitrepo'
# Below is if you have colab pro
REPO = '/root/DLH_TransferLearning/'

In [4]:
%cd $REPO

/root/DLH_TransferLearning


In [5]:
%%capture
! pip install -r requirements.txt

In [6]:
DATA_DIR = PROJECT_ROOT + '/data'

In [7]:
! ls $DATA_DIR

icentia11k		     icentia11k_subset_unzipped		 physionet_finetune
icentia11k_corrupted	     physionet				 physionet_preread
icentia11k_subset	     physionet_250hz_15000pad_norm_True  session_checkpoint.dat
icentia11k_subset_corrupted  physionet_data.zip			 temp.torrent


Set up main job dir that will hold the results of this (and other finetuning) job.

In [8]:
JOB_DIR = PROJECT_ROOT + '/jobs'

**Revision**

We just create a new job subdirectory for this notebook.

> `jobs/finetune_random_cnn_original_data_with_f1`

---
Note that while we did create the `jobs/` directory, we will defer the creation of fine tuning train specific output directory to the trainer code.

So for this experiment, we will write the fine tune results out to `jobs/finetune_random_cnn_original_data`. The name indicates 2 things

1. Random pretrained CNN used
2. We use the same preprocessing steps the authors suggest in their README. Which again is **not** aligned with what they say in the paper.

## Other discrepancies

This is the exact code that the authors say we should run finetuning with:

```shell script
python -m finetuning.trainer \
--job-dir "jobs/af_classification" \
--train "data/physionet_train.pkl" \
--test "data/physionet_test.pkl" \
--weights-file "jobs/beat_classification/resnet18.weights" \
--val-size 0.0625 \
--arch "resnet18" \
--batch-size 64 \
--epochs 200
```

The discrepancies:

1. `--val-metric` is NOT specified. The default value is `loss`. The help message describes this parameter as

  > Validation metric used to find the best model at each epoch.

  However, in the paper, the authors say that they use macro F1 score to evaluate on validation set and also to select the best model.

## Reproducibility

The trainer code also has a `--seed` parameter, which is not provided in the above code snippet.

For our own benefit, we shall set `--seed 2024` for reproducibility in *our* own work.

In [11]:
job_dir = JOB_DIR + '/finetune_random_cnn_original_data_with_f1'
train = DATA_DIR + '/physionet_finetune/physionet_train.pkl'
test = DATA_DIR + '/physionet_finetune/physionet_test.pkl'

print(f"job_dir: {job_dir}")
print(f"train: {train}")
print(f"test: {test}")

job_dir: /content/drive/MyDrive/DLHProject/jobs/finetune_random_cnn_original_data_with_f1
train: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_train.pkl
test: /content/drive/MyDrive/DLHProject/data/physionet_finetune/physionet_test.pkl


Now we use the same settings Myles ran when he tried the precursor notebook. Namely:

1. V100 GPU (16 gb)
2. batch 128

---

**NEW** for this pass, we keep `--epochs=10` initially to see if my changes are working. Namely do we get macro f1 score?


In [12]:
%%time
# We've removed --weights-file parameter.
# We've set --val-metric to f1
# We've set --seed to 2024
# We've set --verbose to see what's going on
! python -m finetuning.trainer \
--job-dir $job_dir \
--train $train \
--test $test \
--val-size 0.0625 \
--val-metric "f1" \
--arch "resnet18" \
--batch-size 128 \
--epochs 10 \
--seed 2024 \
--verbose

2024-04-11 06:20:06.506652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 06:20:06.506746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 06:20:06.508684: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 06:20:06.519572: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 06:20:08.019033: W tensorflow/comp

It took 5m 14s to run 10 epochs